In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import igraph as ig
import os
import yaml
from os.path import join
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
from scipy.stats import entropy
from scipy import stats
from functools import reduce
import sys
sys.path.insert(1, '..')
from utils import *
from collections import Counter
import geopandas as gpd
import matplotlib.ticker as ticker
import geopandas as gpd

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
FIG_SAVE_DIR = join('..', '..', 'figures')
RESULT_SAVE_DIR = join('..', '..', 'results')

In [3]:
with open(join('..', '..', 'config.yml')) as f:
    config = yaml.safe_load(f)
    
pwd = config['pwd']
dpath = config['storage']
patterns = config['pattern-data-path']

AREA_NAME = 'NYC'

In [ ]:
nyc_poi = gpd.read_file()

In [7]:
borough_df = gpd.read_file(join(pwd, 'util_datasets', 'NYC', 'NYC-borough.geojson'))

In [11]:
df = pd.read_csv(join(pwd, 'util_datasets', AREA_NAME, f'{AREA_NAME}-POI-INFO.csv'))
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

In [12]:
merged = gpd.sjoin(gdf, borough_df, how='left', op='within')

In [47]:
sl_poi = merged[merged['boro_name'] == 'Staten Island']['top_category'].value_counts(normalize=False)[::-1]
ax = sl_poi.plot.barh(figsize=(5, 15))
ax.grid(axis='x')
#ax.set_xlim(0, 75)
ax.set_title('Staten Island POI Distribution - Top Categories', fontsize=15)

for i, val in enumerate(sl_poi):
    ax.text(val, i, str(val), verticalalignment='center')
    
#plt.tight_layout()
plt.savefig(join(FIG_SAVE_DIR, 'nyc-staten-island-poi-dist-topcat.png'), bbox_inches='tight')

In [48]:
PATTERN_DIR = join(pwd, dpath, 'filtered-patterns')

AREA_NAME = 'NYC'
FILTERED_PATTERN_DIR = join(PATTERN_DIR, f'{AREA_NAME}-patterns')
pattern_files = [join(FILTERED_PATTERN_DIR, f) for f in os.listdir(FILTERED_PATTERN_DIR) if f.endswith('csv')]

In [56]:
sl_cbgs = set(pd.read_csv(join(pwd, 'util_datasets', 'staten_island_cbgs.csv'))['cbgs'].astype(np.int64).values)

In [57]:
for pattern_file in tqdm(pattern_files):
    df = pd.read_csv(pattern_file)
    df = df[df['poi_cbg'].isin(sl_cbgs)]
    print(df.shape)
    break

  0%|                                                                                          | 0/107 [00:00<?, ?it/s]

(3989, 2)


In [54]:
df.dtypes

poi_cbg               int64
visitor_home_cbgs    object
dtype: object